In [ ]:
# NumPy：数値計算ライブラリ（ここではランダムな数値生成に使用）
import numpy as np

# pandas：表形式データ（DataFrame）を扱うライブラリ（後で.obsや.varに使う）
import pandas as pd

# NumPy：数値計算ライブラリ（ここではランダムな数値生成に使用）
import numpy as np

# pandas：表形式データ（DataFrame）を扱うライブラリ（後で.obsや.varに使う）
import pandas as pd

# csr_matrix：疎行列（ゼロが多い行列）を効率よく扱うための形式
from scipy.sparse import csr_matrix

# anndata：AnnDataオブジェクトを扱うメインライブラリ
import anndata as ad

# 100細胞 × 2000遺伝子の発現データをランダムに生成（ポアソン分布に従う）
# → 値の大部分が0なので、疎行列として保存してメモリを節約
counts = csr_matrix(np.random.poisson(1, size=(100, 2000)), dtype=np.float32)

# AnnDataオブジェクトの作成（X に遺伝子発現行列をセット）
adata = ad.AnnData(X=counts)

# 結果表示：細胞数（obs）と遺伝子数（vars）の形を確認
print(adata)

AnnData object with n_obs × n_vars = 100 × 2000


In [6]:
# 発現行列（カウントデータ）の本体を表示
# → 今はcsr_matrix（スパース行列）形式で保存されている
adata.X
# 各細胞にわかりやすい名前（Cell_0, Cell_1, ..., Cell_99）をつける
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]

# 各遺伝子にもわかりやすい名前（Gene_0, Gene_1, ..., Gene_1999）をつける
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]

# 細胞名の先頭10個を確認
print(adata.obs_names[:10])

Index(['Cell_0', 'Cell_1', 'Cell_2', 'Cell_3', 'Cell_4', 'Cell_5', 'Cell_6',
       'Cell_7', 'Cell_8', 'Cell_9'],
      dtype='object')


In [7]:
# 各細胞（観測行）に細胞タイプのラベルを追加する
# "T細胞" または "B細胞" のどちらかをランダムに割り当てる
# → 結果は adata.obs["cell_type"] に保存される（pandasのDataFrame列として扱える）
adata.obs["cell_type"] = np.random.choice(["T", "B"], size=adata.n_obs)

# 各遺伝子（列）に名前（シンボル）を付ける
# 例：gene0, gene1, ..., gene1999 のような名前が gene_symbols に入る
# → adata.var["gene_symbols"] に格納（遺伝子のメタデータ）
adata.var["gene_symbols"] = [f"gene{i}" for i in range(adata.n_vars)]

# 細胞情報の確認（先頭5行を表示）
# → DataFrame形式で .obs に入っている情報を見ることができる
adata.obs.head()

,cell_type
Cell_0,B
Cell_1,B
Cell_2,T
Cell_3,B
Cell_4,B


In [8]:
# 正規化（Zスコア）：平均0、分散1にスケーリング
adata.layers["zscore"] = (adata.X - adata.X.mean()) / adata.X.std()

# どんなレイヤーがあるか確認
adata.layers


NotImplementedError: subtracting a nonzero scalar from a sparse array is not supported